In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,70,2024-03-22,7,121.000000,96.846377,0.578246,10.311594,24.753623,0.191188,124.230435,...,13.108696,24.668116,0.197565,111.620290,0,3,4,2,2,2024
1,69,2024-03-22,3,120.705882,99.023529,0.576691,10.436765,20.763235,0.203059,121.172059,...,11.540580,24.571014,0.180855,114.146377,0,3,4,2,21,2024
2,71,2024-03-22,-4,106.028571,97.407143,0.513500,12.831429,22.884286,0.187200,108.265714,...,12.642029,22.988406,0.174159,111.178261,0,3,4,2,15,2024
3,70,2024-03-22,-1,113.376812,96.559420,0.557406,11.559420,23.481159,0.184304,116.618841,...,12.311594,23.189855,0.214449,116.715942,0,3,4,2,6,2024
4,69,2024-03-22,3,115.970588,97.461765,0.558926,11.354412,25.342647,0.206132,118.852941,...,11.063768,21.855072,0.214217,114.479710,0,3,4,3,19,2024
5,71,2024-03-22,1,122.814286,101.264286,0.578557,11.004286,23.274286,0.178457,121.087143,...,11.851471,27.338235,0.179529,118.769118,0,3,4,2,12,2024
6,69,2024-03-22,1,116.470588,96.488235,0.566515,11.264706,24.180882,0.215235,120.498529,...,12.756522,26.936232,0.191507,110.359420,0,3,4,2,13,2024
7,70,2024-03-22,-1,114.652174,97.007246,0.530435,10.053623,24.573913,0.227725,117.773913,...,11.718841,19.420290,0.210174,116.607246,0,3,4,2,23,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1
1,2024-03-22,OKC,TOR,1.060606,10.500000,1
2,2024-03-22,MEM,SAS,3.000000,1.420168,0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1
5,2024-03-22,IND,GSW,2.700000,1.487805,1
6,2024-03-22,LAC,POR,1.100000,7.500000,1
7,2024-03-22,PHI,LAL,3.400000,1.335570,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1
1,2024-03-22,OKC,TOR,1.060606,10.500000,1
2,2024-03-22,MEM,SAS,3.000000,1.420168,0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1
5,2024-03-22,IND,GSW,2.700000,1.487805,0
6,2024-03-22,LAC,POR,1.100000,7.500000,1
7,2024-03-22,PHI,LAL,3.400000,1.335570,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1
1,2024-03-22,OKC,TOR,1.060606,10.500000,1
2,2024-03-22,MEM,SAS,3.000000,1.420168,0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0
4,2024-03-22,NOP,MIA,2.300000,1.645161,0
5,2024-03-22,IND,GSW,2.700000,1.487805,0
6,2024-03-22,LAC,POR,1.100000,7.500000,1
7,2024-03-22,PHI,LAL,3.400000,1.335570,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1
1,2024-03-22,OKC,TOR,1.060606,10.500000,1
2,2024-03-22,MEM,SAS,3.000000,1.420168,0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1
5,2024-03-22,IND,GSW,2.700000,1.487805,0
6,2024-03-22,LAC,POR,1.100000,7.500000,1
7,2024-03-22,PHI,LAL,3.400000,1.335570,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1.0
1,2024-03-22,OKC,TOR,1.060606,10.500000,1.0
2,2024-03-22,MEM,SAS,3.000000,1.420168,0.0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0.0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1.0
5,2024-03-22,IND,GSW,2.700000,1.487805,1.0
6,2024-03-22,LAC,POR,1.100000,7.500000,1.0
7,2024-03-22,PHI,LAL,3.400000,1.335570,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-22,BOS,DET,1.083333,8.500000,1.0
1,2024-03-22,OKC,TOR,1.060606,10.500000,1.0
2,2024-03-22,MEM,SAS,3.000000,1.420168,0.0
3,2024-03-22,CLE,MIN,3.350000,1.344828,0.0
4,2024-03-22,NOP,MIA,2.300000,1.645161,1.0
5,2024-03-22,IND,GSW,2.700000,1.487805,1.0
6,2024-03-22,LAC,POR,1.100000,7.500000,1.0
7,2024-03-22,PHI,LAL,3.400000,1.335570,0.0
